In [1]:
import sys
sys.path.append("../../AlphaPose_masterBranch")

In [2]:
import alphapose
from alphapose.utils.detector import DetectionLoader
from alphapose.utils.config import update_config
from easydict import EasyDict as edict
import json
from detector.apis import get_detector
from tqdm import tqdm



In [3]:
cfg = update_config("/home/chen/workshop/thesis/AlphaPose_masterBranch/configs/coco/resnet/256x192_res50_lr1e-3_1x_simplebaseline_crowdpose_test.yaml")


In [4]:
j = json.load(open("/home/chen/workshop/thesis/human-pose-estimation.pytorch/data/crowdpose/annotations/crowdpose_test.json"))

In [5]:
input_source = [str(i['id'])+'.jpg' for i in j['images']]
args = edict()
args.detector="yolo"
args.device='cuda'
args.sp=True
args.gpus=[0]
args.tracking = False
args.inputpath="/home/chen/workshop/thesis/human-pose-estimation.pytorch/data/crowdpose/images"
det_loader = DetectionLoader(input_source, get_detector(args), cfg, args, batchSize=16)
det_worker = det_loader.start()

In [6]:
d = []

template = {
    'bbox': [244.05099093550018,
        170.61324112294565,
        74.56380466590753,
        73.94038239037835],
    'category_id': 1,
    'image_id': 532481,
    'score': 1
}

In [7]:
data_len = det_loader.length
print(data_len)
im_names_desc = tqdm(range(data_len), dynamic_ncols=True)
for i in im_names_desc:
    (inps, orig_img, im_name, boxes, scores, ids, cropped_boxes) = det_loader.read()
    if boxes is None:
        print(1)
        continue
    for idx in range(boxes.size(0)):
        t = template.copy()
        t['bbox'] = boxes[idx].numpy().tolist()
        b = t['bbox']
        b[2] -= b[0]
        b[3] -= b[1]
        t['image_id'] = int(im_name.split('.')[0])
        t['score'] = float(scores[idx])
        d.append(t)


  0%|          | 0/8000 [00:00<?, ?it/s]8000
Loading YOLO model..
100%|██████████| 8000/8000 [21:39<00:00,  6.16it/s]


In [8]:
det_loader.stop()

In [10]:
del det_loader

In [9]:
json.dump(d, open("/home/chen/workshop/thesis/human-pose-estimation.pytorch/data/crowdpose/person_detection_result/test_box.json", 'w'))

In [16]:
d = json.load(open("/home/chen/workshop/thesis/human-pose-estimation.pytorch/data/crowdpose/person_detection_result/test_box.json"))

JSONDecodeError: Expecting value: line 1 column 138 (char 137)

In [11]:
for i in d:
    b = i['bbox']
    b[2] -= b[0]
    b[3] -= b[1]

{'bbox': [106.32122039794922,
  10.575272560119629,
  376.0461730957031,
  370.5619201660156],
 'category_id': 1,
 'image_id': 106848,
 'score': 0.9993866682052612}